In [ ]:
#| default_exp xml

# XML

> Concise generation of XML.

In [ ]:
#| export
from fastcore.utils import *

import types,json

from dataclasses import dataclass, asdict
from typing import Mapping
from functools import partial
from html import escape

In [ ]:
from IPython.display import Markdown
from pprint import pprint

In [ ]:
#|export
class FT:
    "A 'Fast Tag' structure, containing `tag`,`children`,and `attrs`"
    def __init__(self, tag:str, cs:tuple, attrs:dict=None, void_=False, **kwargs):
        assert isinstance(cs, tuple)
        self.tag,self.children,self.attrs,self.void_ = tag,cs,attrs,void_

    def __setattr__(self, k, v):
        if k.startswith('__') or k in ('tag','children','attrs','void_'): return super().__setattr__(k,v)
        self.attrs[k.lstrip('_').replace('_', '-')] = v

    def __getattr__(self, k):
        if k.startswith('__'): raise AttributeError(k)
        return self.get(k)

    @property
    def list(self): return [self.tag,self.children,self.attrs]
    def get(self, k, default=None): return self.attrs.get(k.lstrip('_').replace('_', '-'), default)
    def __repr__(self): return f'{self.tag}({self.children},{self.attrs})'

    def __add__(self, b):
        self.children = self.children + tuplify(b)
        return self
    
    def __getitem__(self, idx): return self.children[idx]
    def __iter__(self): return iter(self.children)

In [ ]:
#| export
_specials = set('@.-!~:[](){}$%^&*+=|/?<>,`')

def attrmap(o):
    if o=='_' or (_specials & set(o)): return o
    o = dict(htmlClass='class', cls='class', _class='class', klass='class',
             _for='for', fr='for', htmlFor='for').get(o, o)
    return o.lstrip('_').replace('_', '-')

In [ ]:
#| export
def valmap(o):
    if is_listy(o): return ' '.join(map(str,o)) if o else None
    if isinstance(o, dict): return '; '.join(f"{k}:{v}" for k,v in o.items()) if o else None
    return o

In [ ]:
#| export
def _flatten_tuple(tup):
    if not any(isinstance(item, tuple) for item in tup): return tup
    result = []
    for item in tup:
        if isinstance(item, tuple): result.extend(item)
        else: result.append(item)
    return tuple(result)

In [ ]:
#|export
def _preproc(c, kw, attrmap=attrmap, valmap=valmap):
    if len(c)==1 and isinstance(c[0], (types.GeneratorType, map, filter)): c = tuple(c[0])
    attrs = {attrmap(k.lower()):valmap(v) for k,v in kw.items() if v is not None}
    return _flatten_tuple(c),attrs

In [ ]:
#| export
def ft(tag:str, *c, void_:bool=False, attrmap:callable=attrmap, valmap:callable=valmap, **kw):
    "Create an `FT` structure for `to_xml()`"
    return FT(tag.lower(),*_preproc(c,kw,attrmap=attrmap, valmap=valmap), void_=void_)

In [ ]:
#| export
voids = set('area base br col command embed hr img input keygen link meta param source track wbr !doctype'.split())
_g = globals()
_all_ = ['Head', 'Title', 'Meta', 'Link', 'Style', 'Body', 'Pre', 'Code',
    'Div', 'Span', 'P', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'Strong', 'Em', 'B',
    'I', 'U', 'S', 'Strike', 'Sub', 'Sup', 'Hr', 'Br', 'Img', 'A', 'Link', 'Nav',
    'Ul', 'Ol', 'Li', 'Dl', 'Dt', 'Dd', 'Table', 'Thead', 'Tbody', 'Tfoot', 'Tr',
    'Th', 'Td', 'Caption', 'Col', 'Colgroup', 'Form', 'Input', 'Textarea',
    'Button', 'Select', 'Option', 'Label', 'Fieldset', 'Legend', 'Details',
    'Summary', 'Main', 'Header', 'Footer', 'Section', 'Article', 'Aside', 'Figure',
    'Figcaption', 'Mark', 'Small', 'Iframe', 'Object', 'Embed', 'Param', 'Video',
    'Audio', 'Source', 'Canvas', 'Svg', 'Math', 'Script', 'Noscript', 'Template', 'Slot']

for o in _all_: _g[o] = partial(ft, o.lower(), void_=o.lower() in voids)

In [ ]:
a = Body(Div('hi', a=1, b=True, cls=()), P('hi', cls=['a',1], style=dict(a=1,b=2)))
a

body((div(('hi',),{'a': 1, 'b': True, 'class': None}), p(('hi',),{'class': 'a 1', 'style': 'a:1; b:2'})),{})

In [ ]:
a + (P('a'),P('b'))

body((div(('hi',),{'a': 1, 'b': True, 'class': None}), p(('hi',),{'class': 'a 1', 'style': 'a:1; b:2'}), p(('a',),{}), p(('b',),{})),{})

The main HTML tags are exported as `ft` partials.

Attributes are passed as keywords. Use 'klass' and 'fr' instead of 'class' and 'for', to avoid Python reserved word clashes.

In [ ]:
#| export
def Html(*c, doctype=True, **kwargs)->FT:
    "An HTML tag, optionally preceeded by `!DOCTYPE HTML`"
    res = ft('html', *c, **kwargs)
    if not doctype: return res
    return (ft('!DOCTYPE', html=True, void_=True), res)

In [ ]:
samp = Html(
    Head(Title('Some page')),
    Body(Div('Some text\nanother line', (Input(name="jph's"), Img(src="filename", data=1)),
             cls=['myclass', 'another'],
             style={'padding':1, 'margin':2}))
)
pprint(samp)

(!doctype((),{'html': True}),
 html((head((title(('Some page',),{}),),{}), body((div(('Some text\nanother line', input((),{'name': "jph's"}), img((),{'src': 'filename', 'data': 1})),{'class': 'myclass another', 'style': 'padding:1; margin:2'}),),{})),{}))


The three elements of the list can also be accessed with property names, so you don't have to remember their order.

In [ ]:
elem = P('Some text', id="myid")
print(elem.tag)
print(elem.children)
print(elem.attrs)

p
('Some text',)
{'id': 'myid'}


You can also get and set attrs directly:

In [ ]:
elem.id = 'newid'
print(elem.id, elem.get('id'), elem.get('foo', 'missing'))
elem

newid newid missing


p(('Some text',),{'id': 'newid'})

In [ ]:
#| export
class Safe(str):
    def __html__(self): return self

In [ ]:
#| export
def _escape(s): return '' if s is None else s.__html__() if hasattr(s, '__html__') else escape(s) if isinstance(s, str) else s

In [ ]:
#| export
def _to_attr(k,v):
    if isinstance(v,bool):
        if v==True : return str(k)
        if v==False: return ''
    if isinstance(v,str): v = escape(v, quote=False)
    elif isinstance(v, Mapping): v = json.dumps(v)
    else: v = str(v)
    qt = '"'
    if qt in v:
        qt = "'"
        if "'" in v: v = v.replace("'", "&#39;")
    return f'{k}={qt}{v}{qt}'

In [ ]:
#| export
def _to_xml(elm, lvl, indent:bool):
    nl = '\n'
    if not indent: lvl,nl = 0,''
    if elm is None: return ''
    if hasattr(elm, '__ft__'): elm = elm.__ft__()
    if isinstance(elm, tuple): return f'{nl}'.join(to_xml(o, indent=indent) for o in elm)
    if isinstance(elm, bytes): return elm.decode('utf-8')
    sp = ' ' * lvl
    if not isinstance(elm, FT): return f'{_escape(elm)}{nl}'

    tag,cs,attrs = elm.list
    stag = tag
    if attrs:
        sattrs = (_to_attr(k,v) for k,v in attrs.items())
        stag += ' ' + ' '.join(sattrs)

    isvoid = getattr(elm, 'void_', False)
    cltag = '' if isvoid else f'</{tag}>'
    if not cs: return f'{sp}<{stag}>{cltag}{nl}'
    if len(cs)==1 and not isinstance(cs[0],(list,tuple)) and not hasattr(cs[0],'__ft__'):
        return f'{sp}<{stag}>{_escape(cs[0])}{cltag}{nl}'
    res = f'{sp}<{stag}>{nl}'
    res += ''.join(to_xml(c, lvl=lvl+2, indent=indent) for c in cs)
    if not isvoid: res += f'{sp}{cltag}{nl}'
    return Safe(res)

def to_xml(elm, lvl=0, indent:bool=True):
    "Convert `ft` element tree into an XML string"
    return Safe(_to_xml(elm, lvl, indent))

FT.__html__ = to_xml

In [ ]:
h = to_xml(samp)
print(h)

<!doctype html>

<html>
  <head><title>Some page</title>
</head>
  <body><div class="myclass another" style="padding:1; margin:2">
Some text
another line
  <input name="jph's">
  <img src="filename" data="1">
</div>
</body>
</html>



In [ ]:
class PageTitle:
    def __ft__(self): return H1("Hello")

class HomePage:
    def __ft__(self): return Div(PageTitle(), Div('hello'))

h = to_xml(Div(HomePage()))
expected_output = """<div>
  <div>
    <h1>Hello</h1>
    <div>hello</div>
  </div>
</div>
"""
assert h == expected_output

In [ ]:
h = to_xml(samp, indent=False)
print(h)

<!doctype html><html><head><title>Some page</title>
</head><body><div class="myclass another" style="padding:1; margin:2">
Some text
another line
  <input name="jph's">
  <img src="filename" data="1">
</div>
</body></html>


Interoperability both directions with Django and Jinja using the [__html__() protocol](https://jinja.palletsprojects.com/en/3.1.x/templates/#jinja-filters.escape):

In [ ]:
def _esc(s): return s.__html__() if hasattr(s, '__html__') else Safe(escape(s))

r = Safe('<b>Hello from Django</b>')
print(to_xml(Div(r)))
print(_esc(Div(P('Hello from fastcore <3'))))

<div><b>Hello from Django</b></div>

<div><p>Hello from fastcore &lt;3</p>
</div>



In [ ]:
#| export
def highlight(s, lang='html'):
    "Markdown to syntax-highlight `s` in language `lang`"
    return f'```{lang}\n{to_xml(s)}\n```'

In [ ]:
#| export
def showtags(s):
    return f"""<code><pre>
{escape(to_xml(s))}
</code></pre>"""

FT._repr_markdown_ = highlight

In [ ]:
#| export
def __getattr__(tag):
    if tag.startswith('_') or tag[0].islower(): raise AttributeError
    def _f(*c, target_id=None, **kwargs): return ft(tag, *c, target_id=target_id, **kwargs)
    return _f

In [ ]:
#|export
@patch
def __call__(self:FT, *c, **kw):
    c,kw = _preproc(c,kw)
    if c: self = self+c
    if kw: self.attrs = {**self.attrs, **kw}
    return self

You can also reorder the children to come *after* the attrs, if you use this alternative syntax for `FT` where the children are in a second pair of `()` (behind the scenes this is because `FT` implements `__call__` to add children).

In [ ]:
Body(klass='myclass')(
    Div(style='padding:3px')(
        'Some text ',
        I(spurious=True)('in italics'),
        Input(name='me'),
        Img(src="filename", data=1)
    )
)

```html
<body class="myclass"><div style="padding:3px">
Some text 
  <i spurious>in italics</i>
  <input name="me">
  <img src="filename" data="1">
</div>
</body>

```

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()